<a href="https://colab.research.google.com/github/munisdev86/fastai-dl-pt1/blob/main/intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>